## Ensemble learning for activity recognition

### 1. Set libraries

In [3]:
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
import keras as keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras import optimizers
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.layers import GRU, LSTM, Activation, Bidirectional
from keras.layers import Dense, Embedding
from keras.layers import TimeDistributed
from keras.layers import Concatenate, Flatten 
from keras.layers import Dense, concatenate
import time
import matplotlib.pyplot as plt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from numpy import dstack
#from attention import Attention

# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import ConvLSTM2D
from scipy.optimize import differential_evolution
from sklearn.datasets import make_blobs

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
import os

### 2. Load the data

In [ ]:
# load dataset with pandas
dataframe1 = pd.read_csv('data/input_v2/Presentation_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Chatting_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Discussion_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/GroupStudy_activity.csv', sep=',', header=None)

# Show how many episodes 
#print(len(dataframe1))
#print(len(dataframe2))
#print(len(dataframe3))
#print(len(dataframe4))

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values

#288, 6
#Seperate input and do one-hot encoding
#X=to_categorical(dataset)
X=dataset
print("The shape of X:", X.shape)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array
y_df = pd.DataFrame({'': np_array[:, 0]})

Y=to_categorical(y_df)
#Y=y_df
print("The shape of Y:", Y.shape)

## 3. Set parameters

In [ ]:
epochs = 50
batch_size = 64
length=724
max_len=length
n_members = 4
n_features = 1
n_seq = 4
n_steps = 181
file_name ='Result_Weighted.csv'

## 4. Construct base models

In [5]:
def model1_made():
    model1 = tf.keras.Sequential()
    model1.add(tf.keras.layers.Embedding(100, 128, input_length=max_len, mask_zero = True))
    model1.add(tf.keras.layers.SpatialDropout1D(0.2))
    model1.add(tf.keras.layers.Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
    model1.add(tf.keras.layers.Dense(4, activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model1

def model2_made():
    model2 = tf.keras.Sequential()
    model2.add(tf.keras.layers.Embedding(100, 128, input_length=max_len, mask_zero = True))
    model2.add(tf.keras.layers.SpatialDropout1D(0.2))  
    model2.add(tf.keras.layers.LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model2.add(tf.keras.layers.Dense(4, activation='softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model2

In [6]:
def model3_made():
    model3 = Sequential()
    model3.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model3.add(SpatialDropout1D(0.2))  
    model3.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
    model3.add(Dense(4, activation='softmax'))
    model3.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model3
def model4_made():
    model4 = Sequential()
    model4.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model4.add(SpatialDropout1D(0.2))  
    model4.add(Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2)))
    model4.add(Dense(4, activation='softmax'))
    model4.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model4

In [7]:
def model5_made():#CNN+BiLSTM
    # define model
    model5 = Sequential()
    model5.add(TimeDistributed(Conv1D(filters=4, kernel_size=1, activation='relu'), 
                               input_shape=(None, n_steps, n_features)))
    model5.add(TimeDistributed(MaxPooling1D(pool_size=4)))
    model5.add(TimeDistributed(Flatten()))
    model5.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
    model5.add(Dense(4, activation='softmax'))
    model5.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model5
def model6_made():#CNN+LSTM
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=4, kernel_size=1, activation='relu'), 
                              input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

def model7_made():#ConvLSTM
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=4, kernel_size=(1,2), activation='relu', 
                         input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(TimeDistributed(Flatten()))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    #model.summary()
    return model

In [7]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

def model8_made(): #BiLSTM with attention
    sequence_input = tf.keras.Input(shape=(max_len,), dtype='int32')
    embedded_sequences = tf.keras.layers.Embedding(100, 128, input_length=max_len, mask_zero = True)(sequence_input)
    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
    lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional \
  (tf.keras.layers.LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)
    print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)
    state_h = tf.keras.layers.Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = tf.keras.layers.Concatenate()([forward_c, backward_c]) # 셀 상태
    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)
    dense1 = tf.keras.layers.Dense(20, activation="relu")(context_vector)
    dropout = tf.keras.layers.Dropout(0.2)(dense1)
    output = tf.keras.layers.Dense(4, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=sequence_input, outputs=output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

def model9_made(): #LSTM with attention
    sequence_input = tf.keras.Input(shape=(max_len,), dtype='int32')
    embedded_sequences = tf.keras.layers.Embedding(100, 128, input_length=max_len, mask_zero = True)(sequence_input)
    lstm = tf.keras.layers.LSTM(64, dropout=0.5, return_sequences = True)(embedded_sequences)
    lstm, forward_h, forward_c = tf.keras.layers.LSTM(64, dropout=0.5, return_sequences=True, return_state=True)(lstm)
    print(lstm.shape, forward_h.shape, forward_c.shape)
    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, forward_h)
    dense1 = tf.keras.layers.Dense(20, activation="relu")(context_vector)
    dropout = tf.keras.layers.Dropout(0.2)(dense1)
    output = tf.keras.layers.Dense(4, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=sequence_input, outputs=output)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

## Contstruct evalution methods

In [8]:
def model_evaluate1(X_train,Y_train,model,X_test, Y_test):
    #tf.debugging.set_log_device_placement(True)
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # evaluate model
    accr = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    #plt.title('Loss')
    #plt.plot(history.history['loss'], label='train')
    #plt.plot(history.history['val_loss'], label='test')
    #plt.legend()
    #plt.show();

    #plt.title('Accuracy')
    #plt.plot(history.history['accuracy'], label='train')
    #plt.plot(history.history['val_accuracy'], label='test')
    #plt.legend()
    #plt.show();

    yhat=model.predict(X_test,verbose=0)
    write_rs(argmax(Y_test,axis=1), argmax(yhat,axis=1))
    #argmax(yhat1, axis=1)
    return [accr, yhat]

In [ ]:
def model_evaluate2(X_train,Y_train,model,X_test, Y_test):
    # fit model
    X_train1 = X_train.reshape((X_train.shape[0], n_seq, n_steps, n_features))
    model.fit(X_train1, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # demonstrate prediction    
    X_test1 = X_test.reshape((X_test.shape[0], n_seq, n_steps, n_features))
    accr = model.evaluate(X_test1, Y_test, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    yhat = model.predict(X_test1, verbose=0)
    write_rs(argmax(Y_test,axis=1), argmax(yhat,axis=1))
    
    return [accr, yhat]
def model_evaluate3(X_train,Y_train,model,X_test, Y_test):    
    # fit model
    X_train2 = X_train.reshape((X_train.shape[0], n_seq, 1, n_steps, n_features))
    Y_train2 = Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))
    model.fit(X_train2, Y_train2, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # demonstrate prediction    
    X_test2 = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
    Y_test2 = Y_test.reshape((Y_test.shape[0], 1, Y_test.shape[1]))
    accr = model.evaluate(X_test2, Y_test2, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
        
    yhat = model.predict(X_test2, verbose=0)
    yhat = np.reshape(yhat,(len(yhat),4))
    write_rs(argmax(Y_test,axis=1), argmax(yhat,axis=1))
    return [accr, yhat]